## Is it a guinea pig, a hamster or something else?

fork from https://www.kaggle.com/code/jhoward/is-it-a-bird-creating-a-model-from-your-own-data

In [1]:
#NB: Kaggle requires phone verification to use the internet or a GPU. If you haven't done that yet, the cell below will fail
#    This code is only here to check that your internet is enabled. It doesn't do anything else.
#    Here's a help thread on getting your phone number verified: https://www.kaggle.com/product-feedback/135367

import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

In [ ]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
try:
    import google.colab
    isColab = True
except:
    isColab = False

if iskaggle:
    !pip install fastai duckduckgo_search
elif isColab:
    !pip install -Uqq fastai
    !pip install -Uqq duckduckgo_search
else:
    !pip install -U fastai
    !pip install -U duckduckgo_search

## Step 1: Download images of guinea pigs, hamsters and other rodents

In [3]:
from fastcore.all import *
from duckduckgo_search import ddg_images

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

Our searches seem to be giving reasonable results, so let's grab a few examples of each of "guinea pig", "hamster" and "rat" photos, and save each group of photos to a different folder (I'm also trying to grab a range of lighting conditions here):

In [ ]:
rodents = 'chinchilla','gerbil','guinea pig','hamster','mouse','octodon','rat','squirrel', 'not-a-rodent'
path = Path('data/rodents')
from time import sleep
from fastai.vision.all import *

def download_images_and_sleep(dest, term, sleep_time=10):
    download_images(dest, urls=search_images(term))
    sleep(10)

for output_value in rodents:
    search_term = output_value
    if (output_value == 'not-a-rodent'):
        search_term = 'cat'
    dest = (path/output_value)
    dest.mkdir(exist_ok=True, parents=True)
    download_images_and_sleep(dest, f'{search_term} photo')
    download_images_and_sleep(dest, f'{search_term} sun photo')
    download_images_and_sleep(dest, f'{search_term} shade photo')
    resize_images(path/output_value, max_size=400, dest=path/output_value)

## Step 2: Train our model

Some photos might not download correctly which could cause our model training to fail, so we'll remove them:

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

To train a model, we'll need `DataLoaders`, which is an object that contains a *training set* (the images used to create a model) and a *validation set* (the images used to check the accuracy of a model -- not used during training). In `fastai` we can create that easily using a `DataBlock`, and view sample images from it:

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=12, nrows=4)

Here what each of the `DataBlock` parameters means:

    blocks=(ImageBlock, CategoryBlock),

The inputs to our model are images, and the outputs are categories (in this case, "bird" or "forest").

    get_items=get_image_files, 

To find all the inputs to our model, run the `get_image_files` function (which returns a list of all image files in a path).

    splitter=RandomSplitter(valid_pct=0.2, seed=42),

Split the data into training and validation sets randomly, using 20% of the data for the validation set.

    get_y=parent_label,

The labels (`y` values) is the name of the `parent` of each file (i.e. the name of the folder they're in, which will be *bird* or *forest*).

    item_tfms=[Resize(192, method='squish')]

Before training, resize each image to 192x192 pixels by "squishing" it (as opposed to cropping it).

Now we're ready to train our model. The fastest widely used computer vision model is `resnet18`. You can train this in a few minutes, even on a CPU! (On a GPU, it generally takes under 10 seconds...)

`fastai` comes with a helpful `fine_tune()` method which automatically uses best practices for fine tuning a pre-trained model, so we'll use that.

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.remove_cb(ProgressCallback)
learn.fine_tune(6)

## Step 3: Use our model (and build your own!)

download a guinea pig image and see if our model can predict it:

In [ ]:
from fastdownload import download_url
# for each item of rodents list except the last one, download a photo of that rodent and predict what it is

categories = learn.dls.vocab
print(f'Categories: {categories}')
for item in rodents[:-1]:
	urls=search_images(f'{item} photo', max_images=1)
	dest = f'data/misc/{item}.jpg'
	download_url(urls[0], dest, show_progress=False)
	im = Image.open(dest)
	im.to_thumb(256,256)
	img = PILImage.create(dest)
	predicted,_,probs = learn.predict(img)
	print(f'Predicted: {predicted}')
	# get index for probs item with the highest probability
	tensor_max_index = probs.argmax()
	max_index = tensor_max_index.item()
	print(f'max_index: {max_index}')
	probability = probs[max_index]
	print(f'Probability float: {probability.item():.04f}')
	if (predicted != rodents[max_index-1]):
		print('predicted field not matching probs max probability!')
	for i in range(len(rodents)-1):
		index_prob = i
		print(f'index_prob: {index_prob}')
		print(f'{i} {rodents[i]}: {probs[index_prob].item():.04f}')
	print(f'Probs: {probs}')